In [6]:
import simpy
import pandas as pd 
import numpy as np 

In [32]:
df = np.random.rand(1000,4)
df = pd.DataFrame(df, dtype=np.float16)
df

c:\Users\klein\miniconda3\envs\master\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,0,1,2,3
0,0.582520,0.430908,0.520996,0.863281
1,0.112610,0.686523,0.980957,0.275635
2,0.348145,0.100403,0.829590,0.535645
3,0.423340,0.261963,0.218872,0.388916
4,0.415283,0.329102,0.240601,0.232178
...,...,...,...,...
995,0.930664,0.081482,0.602539,0.859375
996,0.176514,0.693359,0.780762,0.578613
997,0.760254,0.115173,0.325195,0.456055
998,0.375732,0.972168,0.617676,0.556641


In [33]:
%%timeit

df.loc[df[0] > 0.5].sort_values(0)

199 μs ± 2.35 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
%%timeit

df.sort_values(0).loc[df[0] > 0.5]

304 μs ± 13.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
import random

dcl = []

for a in range(1000):
    dcl.append(
        {
           0: np.random.rand(),
           1: np.random.rand(),
           2: np.random.rand(),
           3: np.random.rand(), 
        }
    )     


In [35]:
%%timeit
list(filter(
    lambda x: x[0] > 0.5, dcl
))

51.3 μs ± 1.86 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [36]:
%%timeit
list(filter(
    lambda x: x[0] > 0.5, sorted(
        dcl, key=lambda y: y[0]
    )
))


140 μs ± 3.44 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [37]:
%%timeit
list(
    sorted(
        filter(
            lambda x: x[0]>5, dcl
        ), key=lambda y: y[0]
    )
)


55 μs ± 2.92 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [45]:
from collections import namedtuple

Machine = namedtuple('Machine', 'size, duration')
m1 = Machine(1, 2)  # Small and slow
m2 = Machine(2, 1)  # Big and fast

env = simpy.Environment()
machine_shop = simpy.FilterStore(env, capacity=2)
machine_shop.items = [m1, m2]  # Pre-populate the machine shop

def user(name, env, ms, size):
    machine = yield ms.get(lambda machine: machine.size == size)
    print(name, 'got', machine, 'at', env.now)
    yield env.timeout(machine.duration)
    # yield ms.put(machine)
    print(name, 'released', machine, 'at', env.now)


users = [env.process(user(i, env, machine_shop, 2))
         for i in range(3)]
env.run()

0 got Machine(size=2, duration=1) at 0
0 released Machine(size=2, duration=1) at 1


In [49]:
a = machine_shop.items
a[0] = Machine(20,2)
machine_shop.items = a
machine_shop.items

[Machine(size=20, duration=2)]

In [50]:
type(a)

list